## Importing packages

In [81]:
library(here, help, pos = 2, lib.loc = NULL) 
source(here('Modules','modules.R'))
#install_all_packages()
load_library_packages()

Error: package or namespace load failed for 'caret' in loadNamespace(i, c(lib.loc, .libPaths()), versionCheck = vI[[i]]):
 there is no package called 'recipes'
Warning message in FUN(X[[i]], ...):
"there is no package called 'klaR'"Warning message in FUN(X[[i]], ...):
"there is no package called 'devtools'"Warning message:
"package 'DataExplorer' was built under R version 3.6.3"Error: package or namespace load failed for 'DataExplorer' in loadNamespace(i, c(lib.loc, .libPaths()), versionCheck = vI[[i]]):
 there is no package called 'rmarkdown'
Warning message in FUN(X[[i]], ...):
"there is no package called 'dummies'"Warning message in FUN(X[[i]], ...):
"there is no package called 'ggfortify'"Warning message in FUN(X[[i]], ...):
"there is no package called 'obliqueRF'"

[[1]]
[1] TRUE

[[2]]
[1] TRUE

[[3]]
[1] TRUE

[[4]]
[1] TRUE

[[5]]
[1] TRUE

[[6]]
[1] FALSE

[[7]]
[1] TRUE

[[8]]
[1] TRUE

[[9]]
[1] TRUE

[[10]]
[1] TRUE

[[11]]
[1] TRUE

[[12]]
[1] TRUE

[[13]]
[1] TRUE

[[14]]
[1] TRUE

[[15]]
[1] FALSE

[[16]]
[1] TRUE

[[17]]
[1] TRUE

[[18]]
[1] TRUE

[[19]]
[1] TRUE

[[20]]
[1] TRUE

[[21]]
[1] FALSE

[[22]]
[1] TRUE

[[23]]
[1] TRUE

[[24]]
[1] TRUE

[[25]]
[1] FALSE

[[26]]
[1] FALSE

[[27]]
[1] TRUE

[[28]]
[1] TRUE

[[29]]
[1] TRUE

[[30]]
[1] TRUE

[[31]]
[1] TRUE

[[32]]
[1] TRUE

[[33]]
[1] TRUE

[[34]]
[1] FALSE

[[35]]
[1] TRUE

[[36]]
[1] FALSE

[[37]]
[1] TRUE

[[38]]
[1] TRUE

[[39]]
[1] TRUE

[[40]]
[1] TRUE

[[41]]
[1] TRUE

## Importing and Cleaning dataset

In [82]:
df_during <- read_excel(path = here('Data','during_dataset.xlsx'))
df_during <- data.frame(df_during)
unique(sapply(df_during, class))
df_during <- df_during[complete.cases(df_during), ]
dim(df_during)
head(df_during)

[1] "numeric"

[1] 1249   46

PA_barrier_time_family_responsabilities,PA_barrier__tiredness,PA_barrier_places_closed_dangerous,PA_barrier_money,PA_barrier_time_convenience,PA_barrier_hard_work,PA_barrier_no_interest,PA_barrier_none,PA_easiness_outdoor_activities,PA_easiness_professional_guidance,...,volleyball,basketball,swimming,surfing,yoga,pilates,fighting,other_sports,depression,TAG
0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
0,0,1,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,1
0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0


### Converting dataset types and labels

- Intesity, duration and frequency >> convert dont practive to zero

In [83]:
df_during$PA_intensity[df_during$PA_intensity == 4] <- 0
df_during$PA_duration[df_during$PA_duration == 5] <- 0
df_during$PA_weekly_frequency[df_during$PA_weekly_frequency == 4] <- 0

- Change practice of exercise boolean logic

In [84]:
df_during$PA_practice_during <- ifelse(df_during$PA_practice_during == 0, 1, 0)

- Change sedentary range to boolean logic

In [85]:
df_during$sedentary_time_range_during[df_during$sedentary_time_range_during < 5] <- 0
df_during$sedentary_time_range_during[df_during$sedentary_time_range_during == 5] <- 1

- Change scholarity range

In [86]:
df_during$scholarity[df_during$scholarity <= 3] <- 1
df_during$scholarity[df_during$scholarity > 3 & df_during$scholarity <= 5] <- 2
df_during$scholarity[df_during$scholarity == 6] <- 3

- Removing sex others

In [87]:
df_during <- df_during[df_during$sex != 3, ]

### Creating a number of exercises column

In [88]:
sports <- c("running","hiking","cycling",
            "muscle_training","soccer","tennis",
            "volleyball","basketball","swimming",
            "surfing","yoga","pilates","fighting","other_sports")

df_during$PA_number <- rowSums(df_during[, sports])

In [89]:
`%ni%` <- Negate(`%in%`)
df_during <- df_during[names(df_during) %ni% sports]

### Removing Strangers

In [90]:
differ_p <- (   df_during$PA_duration == 0 
                | df_during$PA_intensity == 0 
                | df_during$PA_weekly_frequency == 0
                | df_during$PA_number == 0
            )
strange_p <- df_during$PA_practice_during == 1 & differ_p
df_during <- df_during[!strange_p, ]
table(strange_p)

differ_dp <- ( df_during$PA_duration != 0 
                | df_during$PA_intensity != 0 
                | df_during$PA_weekly_frequency != 0
                | df_during$PA_number != 0
            ) 
strange_dp <- df_during$PA_practice_during == 0 & differ_dp 


table(strange_dp)
df_during <- df_during[!strange_dp, ]

strange_p
FALSE  TRUE 
 1240     6 

strange_dp
FALSE  TRUE 
 1214    26 

- reindex

In [91]:
rownames(df_during) <- 1:nrow(df_during)

- changing barrier_none label

In [ ]:
names(df_during)[8] <- 'PA_barrier_percieve'

- convert dtype

In [92]:
df_during[, 1:ncol(df_during)] <- lapply(df_during[, 1:ncol(df_during)], as.factor)

## Creting barriers dataset

### Separing Barriers from dataset
<hr />

In [93]:
df_during[1:18] <- lapply(df_during[1:18], as.factor)
df_during_barriers <- df_during[, 1:8]
dim(df_during_barriers)
head(df_during_barriers)

[1] 1214    8

PA_barrier_time_family_responsabilities,PA_barrier__tiredness,PA_barrier_places_closed_dangerous,PA_barrier_money,PA_barrier_time_convenience,PA_barrier_hard_work,PA_barrier_no_interest,PA_barrier_none
0,0,1,0,0,0,0,0
0,0,1,0,0,0,1,0
0,0,1,0,0,0,0,0
0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0
0,0,1,0,0,1,0,0


### Agregating barriers 
<hr />

In [94]:

time_barrier_1 <- ifelse(df_during_barriers$PA_barrier_time_family_responsabilities=='0',0,1)
time_barrier_2 <- ifelse(df_during_barriers$PA_barrier_time_convenience=='0',0,1)


hard_barrier_1 <- ifelse(df_during_barriers$PA_barrier__tiredness=='0',0,1)
hard_barrier_2 <- ifelse(df_during_barriers$PA_barrier_hard_work=='0',0,1)


df_during_barriers$PA_barrier_time <- (time_barrier_1 | time_barrier_2) + 0 
df_during_barriers$PA_barrier_hard <- (hard_barrier_1 | hard_barrier_2) + 0

In [95]:
df_during_barriers$PA_barrier_time_family_responsabilities <- NULL
df_during_barriers$PA_barrier_time_convenience <- NULL

df_during_barriers$PA_barrier__tiredness <- NULL
df_during_barriers$PA_barrier_hard_work <- NULL

In [96]:
names(df_during_barriers)

[1] "PA_barrier_places_closed_dangerous" "PA_barrier_money"                  
[3] "PA_barrier_no_interest"             "PA_barrier_none"                   
[5] "PA_barrier_time"                    "PA_barrier_hard"

## Creating Facilitators dataset

### Separing facilitators from dataset
<hr />

In [97]:
df_during[1:18] <- lapply(df_during[1:18], as.factor)
df_during_facilitators <- df_during[, 9:18]
dim(df_during_facilitators)
head(df_during_facilitators)

[1] 1214   10

PA_easiness_outdoor_activities,PA_easiness_professional_guidance,PA_easiness_unguided,PA_easiness_music,PA_easiness_collective,PA_easiness_home_activity,PA_easiness_family_activity,PA_easiness_home_space,PA_easiness_home_equipament,PA_easiness_others
1,0,0,0,0,0,0,1,1,0
0,0,1,0,0,0,0,0,0,0
1,1,0,0,0,1,0,1,1,0
0,0,0,0,0,1,1,1,1,0
1,0,0,0,0,0,0,0,1,0
0,1,0,1,0,0,1,0,1,0


### Agregating facilitators
<hr />

In [98]:

home_facilitator_1 <- ifelse(df_during_facilitators$PA_easiness_home_activity=='0',0,1)
home_facilitator_2 <- ifelse(df_during_facilitators$PA_easiness_home_equipament=='0',0,1)
home_facilitator_3 <- ifelse(df_during_facilitators$PA_easiness_home_space=='0',0,1)


group_easiness_1 <- ifelse(df_during_facilitators$PA_easiness_family_activity=='0',0,1)
group_easiness_2 <- ifelse(df_during_facilitators$PA_easiness_collective=='0',0,1)



df_during_facilitators$PA_facilitator_home <- (home_facilitator_1 | home_facilitator_2 | home_facilitator_3) + 0 
df_during_facilitators$PA_facilitator_group<- (group_easiness_1 | group_easiness_2) + 0


In [99]:
df_during_facilitators$PA_easiness_home_activity <- NULL
df_during_facilitators$PA_easiness_home_equipament <- NULL
df_during_facilitators$PA_easiness_home_space <- NULL
df_during_facilitators$PA_easiness_family_activity <- NULL
df_during_facilitators$PA_easiness_collective <- NULL
df_during_facilitators$PA_easiness_unguided <- NULL

In [100]:
names(df_during_facilitators)

[1] "PA_easiness_outdoor_activities"    "PA_easiness_professional_guidance"
[3] "PA_easiness_music"                 "PA_easiness_others"               
[5] "PA_facilitator_home"               "PA_facilitator_group"